# 炼丹杂谈

其实我也不知道到底分享这种东西到底能讲些啥…如果说做paper reading或是专题报告的分享，我大概还能说的具体点。这儿就当先开个坑吧。

除了一些结构上具有开创意义的创新，大家都是小修小补，想发文章就找个差点的baseline飚一飚，打比赛就找最好的部件拼一拼开始加trick开始大炼丹。其实都无外乎网络加深提feature再大batch到小batch交替摩擦。loss不降？查查数据，没问题加几层。上大batch加lr再小batch磨一磨，反正有BN这种东西以后大batch满天飞。所以说系统找个啥炼丹心得可说还真不容易……回头系统整理下再重新写。

不过这倒是可以先谈个问题：你train的真的准么……为啥我问这个问题？你可以自己手写几个层，跟pytorch和tf封装好的层测测输出误差……其实很好奇，为啥pytorch调的cudnn，误差居然比tf大。我有点没搞懂到底谁有问题了。最神奇的还是跟手写的卷积都不一样……如果测试没问题，那还是tensorflow的更准一点，不排除因为浮点精度问题出现了累积误差，但是这确实也挺难搞懂的，回头有时间仔细看看pytorch的卷积怎么写的。

## training技巧

其实就实验科学的结果来说，这东西真的是个玄学。
下面先讲讲我自己的实验习惯，解释的话可能过段时间再详细开一篇聊聊。
首先我们讲一讲选模型的原则。首先如果需要深层次的特征，抛开任务，提特征的时候有人都会偏爱直接选择resnet之类的比较深比较大的网络，但是很多时候在试验的情况下发现loss一直不降，震荡，不收敛，这时候……
先看看数据集或是喂的数据有没有问题才是第一要务。先检查自己prepare的数据是否没问题，比如检测自己的crop是不是有问题，或是在其他的一些数据处理函数上可视化view一下。
```
如果本身喂进网络的数据没有问题，检查一下数据集。如果样本数量本来就不多，可以考虑先用类似的小一点模型，比如resneXt置换成GoogLenet，设置好超参数。
如果在较小的网络上这组超参可以收敛，再把这组参数迁移到大模型上，如果还是不收敛或是还在震荡，可以设optimizer为adam，调小一点batch，然后看看能不能在训练中跳出局部极值。跳出去以后可以再选用大batch size和之前在小模型上使用的超参继续训。
```
说到这可能有入坑小伙伴还没搞懂大batch到底有个啥用，或者batch size这个参数到底影响了什么……其实引入batch normalization后，确实大batch带来的收益更诱人了，目前大量的网络结构也加了相当多的bn层，这样使用大batch有了很好的条件。在这我先简单解释下batch size的一些作用或者意义：
- 首先我们谈谈大batch的作用。以sgd举例，事实上sgd大家都知道选一批样本去更新梯度，学这个梯度下降的方向。我们选用大batch，样本更有代表性，因为选择的样本越多，下降肯定越趋于整个数据集收敛应该朝向的方向，所以大batch会更稳定，基本不会在局部震荡， 因为每次参数更新所用到的数据越多，越能代表整体损失函数的梯度，因此梯度精确度更高。另一点是快，而且可以更好利用矩阵计算库里大矩阵乘法的效率，还可以提高显存利用率。
- 那小batch自然没有上述的好处，取一个极端例子，如果我们把batch size设为1，这样称为在线学习，每次修正方向以各自样本的梯度方向修正，难以达到收敛。可以理解为因为每次梯度更新只用一个样本，所以后果就是很可能在某一个局部极值点附近震荡……同理可辐射到其他小batch size上，但是同样也有好处，可以在达到一定程度后用来精细的去磨一下你的模型。因为小批量在学习过程中加入了噪声，会有一些正则化效果，但是需要小的学习率保持稳定性，否则还是会跳进局部极值难以自拔
- 对于大batch size，达到相同精度需要更多epoch，因为每轮训练中的迭代次数更少。

所以可以总结下，适当增大batch size好处在于：
- 内存利用率提高了。
- 对于相同数据量的处理速度更快。
- 在一定范围内，一般来说batch size 越大，其确定的下降方向越准，引起训练震荡越小。

但是盲目增大绝对不行，毕竟首先不是所有实验室都有那么好的设备，batch size上去了，首先也得有足够内存和显存支撑。
另外根据上面提到的要想达到相同的精度，其所花费的时间大大增加了，从而对参数的修正也就显得更加缓慢。主要还有一点，batch size大到一定程度，下降方向已经基本不再变化，加了也没什么用。
所以说本质上还是需要考虑到你的任务到底塞多少合适，另外你的设备到底能怎么撑。一般来说，取2,4,8这么多块卡并行跑，选择一个合适的batch size塞显存就很重要了，因为要考虑显存占用率，你的实验室基友们也不是不跑实验的……不过一般来说，开始训勇敢取个大batch并没啥坏处，有bn撑腰，一切显得美好很多。
当然了，上面说的一切都要看你的实验室/公司项目组有没有钱，毕竟钱才是科学第一推动力。感谢实验室，感谢以前搭集群的师兄们，真的，实属带善人。（保证我能在核弹水平服务器上钓上一整天的鱼）